In [ ]:
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 16.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skyfield.api import load, EarthSatellite

In [ ]:
def load_tle(tle_file_path):
    satellites = {}
    with open(tle_file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            name = lines[i].strip()
            line1 = lines[i + 1].strip()
            line2 = lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name)
            satellites[name] = satellite
    return satellites

tle_file_path = 'active.txt'
satellites = load_tle(tle_file_path)

In [ ]:
def calculate_features(satellite1, satellite2, time):
    position1 = satellite1.at(time).position.km
    position2 = satellite2.at(time).position.km
    velocity1 = satellite1.at(time).velocity.km_per_s
    velocity2 = satellite2.at(time).velocity.km_per_s

    # Features
    distance = np.linalg.norm(position1 - position2)  # Positional uncertainty
    relative_velocity = np.linalg.norm(velocity1 - velocity2)  # Relative velocity
    inclination1 = satellite1.model.inclo  # Orbital inclination (degrees)
    inclination2 = satellite2.model.inclo
    eccentricity1 = satellite1.model.ecco  # Orbital eccentricity
    eccentricity2 = satellite2.model.ecco

    return [distance, relative_velocity, inclination1, inclination2, eccentricity1, eccentricity2]

In [ ]:
def generate_dataset(satellites, num_samples=500):
    ts = load.timescale()
    time = ts.now()
    data = []

    satellite_names = list(satellites.keys())
    for _ in range(num_samples):
        sat1 = satellites[np.random.choice(satellite_names)]
        sat2 = satellites[np.random.choice(satellite_names)]
        features = calculate_features(sat1, sat2, time)
        collision = np.random.choice([0, 1], p=[0.95, 0.05])  # Simulated label
        data.append(features + [collision])

    columns = [
        "Distance", "Relative_Velocity", "Inclination1", "Inclination2",
        "Eccentricity1", "Eccentricity2", "Collision"
    ]
    return pd.DataFrame(data, columns=columns)

dataset = generate_dataset(satellites)

In [ ]:
# train on random forest model
X = dataset.drop("Collision", axis=1)
y = dataset["Collision"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# evalute
y_pred = model.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98        97
           1       0.00      0.00      0.00         3

    accuracy                           0.97       100
   macro avg       0.48      0.50      0.49       100
weighted avg       0.94      0.97      0.96       100



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# real-time collision prediction
ts = load.timescale()
time = ts.now()

In [ ]:
# predict collision for two satellites
sat1 = satellites[list(satellites.keys())[0]]
sat2 = satellites[list(satellites.keys())[1]]

features = calculate_features(sat1, sat2, time)
collision_prob = model.predict_proba([features])[0][1]
print(f"\nPredicted Collision Probability: {collision_prob:.4f}")


Predicted Collision Probability: 0.1500


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
